# Imports

In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import pickle

# Load key column names

In [2]:
protected_classes = ['Gender',
 'Race (Reported)',
 'Race (OMB)',
 'Ethnicity (Reported)',
 'Ethnicity (OMB)']

genes = ['Cyp2C9 genotypes', 'Genotyped QC Cyp2C9*2', 'Genotyped QC Cyp2C9*3',
       'Combined QC CYP2C9','VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T',
       'VKORC1 QC genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T',
       'VKORC1 genotype: 497T>G (5808); chr16:31013055; rs2884737; A/C',
       'VKORC1 QC genotype: 497T>G (5808); chr16:31013055; rs2884737; A/C',
       'VKORC1 genotype: 1173 C>T(6484); chr16:31012379; rs9934438; A/G',
       'VKORC1 QC genotype: 1173 C>T(6484); chr16:31012379; rs9934438; A/G',
       'VKORC1 genotype: 1542G>C (6853); chr16:31012010; rs8050894; C/G',
       'VKORC1 QC genotype: 1542G>C (6853); chr16:31012010; rs8050894; C/G',
       'VKORC1 genotype: 3730 G>A (9041); chr16:31009822; rs7294;  A/G',
       'VKORC1 QC genotype: 3730 G>A (9041); chr16:31009822; rs7294;  A/G',
       'VKORC1 genotype: 2255C>T (7566); chr16:31011297; rs2359612; A/G',
       'VKORC1 QC genotype: 2255C>T (7566); chr16:31011297; rs2359612; A/G',
       'VKORC1 genotype: -4451 C>A (861); Chr16:31018002; rs17880887; A/C',
       'VKORC1 QC genotype: -4451 C>A (861); Chr16:31018002; rs17880887; A/C',
       'CYP2C9 consensus', 'VKORC1 -1639 consensus', 'VKORC1 497 consensus',
       'VKORC1 1173 consensus', 'VKORC1 1542 consensus',
       'VKORC1 3730 consensus', 'VKORC1 2255 consensus',
       'VKORC1 -4451 consensus']
medication = ['Aspirin', 'Acetaminophen or Paracetamol (Tylenol)',
       'Was Dose of Acetaminophen or Paracetamol (Tylenol) >1300mg/day',
       'Simvastatin (Zocor)', 'Atorvastatin (Lipitor)', 'Fluvastatin (Lescol)',
       'Lovastatin (Mevacor)', 'Pravastatin (Pravachol)',
       'Rosuvastatin (Crestor)', 'Cerivastatin (Baycol)',
       'Amiodarone (Cordarone)', 'Carbamazepine (Tegretol)',
       'Phenytoin (Dilantin)', 'Rifampin or Rifampicin',
       'Sulfonamide Antibiotics', 'Macrolide Antibiotics',
       'Anti-fungal Azoles', 'Herbal Medications, Vitamins, Supplements']

# Load and clean data

In [3]:
import os
os.chdir('data')
iwpc = pd.read_excel('warfarin_iwpc.xls',sheet_name='Subject Data')
iwpc = iwpc.drop(['PharmGKB Subject ID',
                             'PharmGKB Sample ID',
                            'Comments regarding Project Site Dataset',
                            'Comorbidities',
                            'Medications',
                            'Indication for Warfarin Treatment',
                            'Estimated Target INR Range Based on Indication'],axis=1)

iwpc['Age'] = iwpc['Age'].str.split().str[0].str.strip('+').astype(float)+5 #eg converts "50-60" age group to 55
iwpc = iwpc[iwpc['Race (OMB)'].isin(['White','Asian','Black or African American'])] #filters out other races too small to evaluate
iwpc = iwpc[~pd.isna(iwpc['Therapeutic Dose of Warfarin'])] #filters out rows where no therapeutic dose is given
iwpc['target'] = (iwpc['Therapeutic Dose of Warfarin']>35) #dichotomizes into low and high dose
iwpc = iwpc.infer_objects()

float_dtypes = iwpc.select_dtypes(include=['int64','float64']).columns
float_index = [list(iwpc.columns).index(k) for k in float_dtypes]

#converts object datatypes into ordinals
obj_dtypes = list(iwpc.select_dtypes(include=['object']).columns)
for protected_class in protected_classes:
    obj_dtypes.remove(protected_class)

obj_index = [list(iwpc.columns).index(k) for k in obj_dtypes]
for idx in obj_index:
    iwpc[iwpc.columns[idx]] = iwpc[iwpc.columns[idx]].astype(str)
oe = OrdinalEncoder()
iwpc[obj_dtypes] = oe.fit_transform(iwpc[obj_dtypes].astype(str))


race_labels = oe.categories_[4]

#fills remaining nans
iwpc = iwpc.fillna(-1)

In [4]:
primary, secondary = train_test_split(iwpc, test_size=0.5, random_state=42) #creates primary (modelling) dataset, secondary (proxy) dataset
primary = primary[[c for c in primary.columns if c not in protected_classes]]
secondary = secondary[genes+medication+['Race (OMB)']] #includes only suggested proxy variables in secondary set
train, test = train_test_split(primary, test_size=0.5, random_state=42)

In [5]:
exclude_from_train = ['Therapeutic Dose of Warfarin','target','PharmGKB Subject ID','PharmGKB Sample ID','Ethnicity (OMB)',
                      'Race (Reported)',
                     'Ethnicity (Reported)']
y_train = train['target']
y_test = test['target']
X_train = train[[c for c in train.columns if c not in exclude_from_train]]
X_test = test[[c for c in test.columns if c not in exclude_from_train]]

In [6]:
rfc = RandomForestClassifier(random_state = 42)
rfc.fit(X_train.values, y_train)
confusion_matrix(y_test.astype(int), rfc.predict(X_test.values))

array([[764, 112],
       [187, 200]], dtype=int64)

In [7]:
pd.concat([X_test, y_test],axis=1).to_csv('primary.csv')
secondary.to_csv('secondary.csv')
pickle.dump(rfc, open('milestone_4_model.pkl','wb'))